In [1]:
%load_ext autoreload
%autoreload 2
%aimport - pulp
%aimport - pulp.pulp


import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Third-party imports
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from index_travel_accessibility.travel_time_and_centroid import (
    map_predicted_and_existing_hospitals,
)

# Local imports
from ai_planner.planner import (
    HospitalPlanner,
    CRS
)

from index_travel_accessibility.travel_time_and_centroid import (
    get_hospital_df,
)
from main import (
    Index,
    assemble_indexes,
    centroid,
    current_hospital_demand,
    equity_index,
)


### Create Equity Index

In [ ]:
index_df = assemble_indexes()
# Set the weights for the indexes
weight = {
    Index.FORECAST_DEMAND: 0.25,
    Index.ELDERLY_SHARE: 0.25,
    Index.GISD: 0.25,
    Index.HOSPITAL_CAPACITY: 0.25,
    Index.TRAVEL_TIME: 0.25,
    Index.ACCESSIBILITY: 0.25,
}
# equity_df = equity_index(index_df, weight)

/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/stats

ARIMA(3,0,0) AIC=-135.65
Current best order (3,0,0)
ARIMA(1,1,0) AIC=429.95
Current best order (1,1,0)
ARIMA(1,1,0) AIC=432.49
Current best order (1,1,0)
ARIMA(1,1,0) AIC=452.99
Current best order (1,1,0)
ARIMA(1,1,0) AIC=439.17
Current best order (1,1,0)
ARIMA(1,1,0) AIC=419.81
Current best order (1,1,0)
ARIMA(1,1,0) AIC=467.72
Current best order (1,1,0)
[GISD] Wrote 6 Saarland districts to /home/mws/uni/Data Science/equity-aware-geospatial-ai-project/src/index_gisd/data/processed/gisd_norm_saarland.csv


/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


********************************************************************************
Estimated time to run the pipeline ---->  2-3 mins
********************************************************************************
✅ Saved Saarland Hospital Dataset


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Friedenstraße 2, 66822 Lebach, Germany',), **{}).
Traceback (most recent call last):
  File "/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "/home/mws/miniconda3/envs/data_science/lib/python3.13/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
  File "/home/mws/miniconda3/envs/data_science/lib/python3.13/http/client.py", line 1430, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "/home/mws/miniconda3/envs/data_science/lib/python3.13/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "/home/mws/miniconda3/envs/data_science/lib/python3.13/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  

********************************************************************************
✅ Saved Hospital Coordinates Dataset
********************************************************************************
✅ Saved 5 evenly spread sample points per district
********************************************************************************
✅ Found nearest hospitals to all sample points.
********************************************************************************
✅ Generated Travel times from sample points to the nearest hospitals
********************************************************************************
✅ Calculated MEAN, MEDIAN and 95th PERCENTILE of the Travel time for EACH district


In [ ]:
equity_df = index_df[Index.FORECAST_DEMAND].reset_index(drop=True)

### Create Centroids Dataframe

#### Create 1 Centroid per Dataframe

In [3]:

#STEP1: Load and Prepare District Data (Centroids)
curr_hospital_demand = current_hospital_demand()
idx = curr_hospital_demand.index

# Create the districts dataframe without MultiIndex
districts_df = pd.DataFrame({
    "demand": curr_hospital_demand.reindex(idx),
    "equity_index": equity_df.reindex(idx),
    "centroid": centroid().reindex(idx)
})
districts_df.index = districts_df.index.astype(int)
# Create GeoDataFrame properly
districts_gdf = gpd.GeoDataFrame(districts_df, geometry="centroid", crs=CRS)
districts_gdf = districts_gdf.reset_index().rename(columns={'index': 'district_code'})


c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[0] = DISTRICT_CODE
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[1:] = col.iloc[1:].map(lambda x: int(x.split('/')[0]))
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_travel_accessibility\travel_time_and_centroid.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely

#### Create 5 Centroids per District

In [ ]:
# Load the Excel file
file_path = "./../index_travel_accessibility/data/processed/saarland_districts_sample_points.xlsx"
df_sample_points = pd.read_excel(file_path)
SAARLAND_DISTRICT_MAPPING = {
    "Saarlouis": 10044,
    "St. Wendel": 10046,
    "Saarpfalz-Kreis": 10045,
    "Merzig-Wadern": 10042,
    "Neunkirchen": 10043,
    "Regionalverband Saarbrücken": 10041 
}
# Display basic information about the dataframe
df_sample_points = df_sample_points.replace(SAARLAND_DISTRICT_MAPPING)
df_sample_points = df_sample_points.set_index('district')

#STEP1: Load and Prepare District Data (Centroids)
curr_hospital_demand = current_hospital_demand()
idx = curr_hospital_demand.index

# Create the districts dataframe without MultiIndex
districts_df = pd.DataFrame({
    "demand": curr_hospital_demand.reindex(idx),
    "equity_index": equity_df.reindex(idx),
    "centroid": centroid().reindex(idx)
})
districts_df.index = districts_df.index.astype(int)
districts_df = df_sample_points.join(districts_df[['demand', 'equity_index']], how='left')
districts_df["centroid"] = districts_df.apply(
    lambda row: Point(row["longitude"], row["latitude"]),
    axis=1
)
# Create GeoDataFrame properly
districts_gdf = gpd.GeoDataFrame(districts_df, geometry="centroid", crs=CRS)
districts_gdf = districts_gdf.reset_index().rename(columns={'district': 'district_code'})


C:\Users\ACER\AppData\Local\Temp\ipykernel_2856\384420393.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_sample_points = df_sample_points.replace(SAARLAND_DISTRICT_MAPPING)
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col.iloc[0] = DISTRICT_CODE
c:\Users\ACER\Downloads\Project\equity-aware-geospatial-ai-project\src\index_demand_forecast\demand_forecast.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

### Executing Planner

In [ ]:
#STEP1: Generate Candidates and Initialize Graph
n_samples_per_centroid = 10
radius_km = 5
planner = HospitalPlanner(districts_gdf=districts_gdf)
planner.generate_candidates(n_samples_per_centroid=n_samples_per_centroid, radius_km=radius_km)
# planner.init_graph()
# Save the graph file first to avoid re-downloading the graph
# G = ox.graph_from_place("Saarland, Germany", network_type='drive')
# ox.save_graphml(G, "saarland_graph.graphml")
planner.init_graph(graph_path="./../saarland_graph.graphml")

#### Add Both Constraints

In [ ]:
# Get hospital data and convert to GeoDataFrame
hospital_df = get_hospital_df()

# Convert to GeoDataFrame using latitude and longitude columns
hospital_gdf = gpd.GeoDataFrame(
    hospital_df, 
    geometry=gpd.points_from_xy(hospital_df.Lon, hospital_df.Lat),
    crs=CRS  # WGS84 coordinate system
)
planner.init_baseline_model(
    # demand_weight=1, # this is added in the Equity Index so no need to set the weight 
    equity_weight=1,
    travel_weight=0,
    beds_weight=0,
    time_threshold=30, #in minutes
    max_beds_per_hospital=300,
    min_beds_per_hospital=20,
    max_beds=1500
)
planner.add_existing_hospitals_constraints(
    gdf=hospital_gdf,
    supply_weight=0,
    distance_threshold=5000, #in meters
    num_neighbors=2
)
planner.add_neighbor_constraints(
    distance_threshold=5000 #in meters
)   

#### Set the number of prediction

In [ ]:
#STEP3: Set Number of Predictions
planner.set_num_predictions(k=10)

#### Predict and Plot

In [ ]:
selected, assigns, beds = planner.predict()
selected_candidates = planner.candidates_gdf.iloc[selected]
selected_candidates['bed_allocation'] = selected_candidates.index.map(lambda id: beds[id])
# Add Lat and Lon columns using assign
# selected_candidates have Lat and Lon of the new predicted hospitals
selected_candidates = selected_candidates.assign(
    Lat=selected_candidates.geometry.y,
    Lon=selected_candidates.geometry.x
)
map_predicted_and_existing_hospitals(
    'results/maps/demand_based_model.html', 
    selected_candidates
)

c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\geopandas\geodataframe.py:1981: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


✅ Map saved with existing and predicted hospitals!


In [10]:
selected_candidates

,geometry,district_code,node,bed_allocation,Lat,Lon
2,POINT (6.92297 49.29626),10041,264134818,300.0,49.296263,6.922973
19,POINT (7.01124 49.3396),10043,389033151,300.0,49.339599,7.011240
21,POINT (7.08335 49.40006),10043,734652583,200.0,49.400060,7.083352
22,POINT (7.03228 49.35362),10043,1519431680,300.0,49.353624,7.032277
35,POINT (6.81524 49.33791),10044,318113685,300.0,49.337907,6.815235
37,POINT (6.93648 49.47145),10044,737973158,20.0,49.471446,6.936483
48,POINT (6.95821 49.5156),10046,458431589,20.0,49.515596,6.958212
49,POINT (6.97611 49.48659),10046,9396802333,20.0,49.486592,6.976113
50,POINT (7.04376 49.51825),10046,590015491,20.0,49.518248,7.043764
53,POINT (7.07365 49.51433),10046,443462848,20.0,49.514334,7.073653


In [ ]:
# Modifying the existing hospitals DataFrame to Prediction DataFrame format
existing_hospitals = get_hospital_df()

existing_hospitals.rename(columns={
    'MaxBeds': 'bed_allocation',
    'SiteID': 'node'
}, inplace=True)


# Adding Geometry column to existing_hospitals
existing_hospitals['Lat'] = existing_hospitals['Lat'].round(6)
existing_hospitals['Lon'] = existing_hospitals['Lon'].round(6)
existing_hospitals['geometry'] = existing_hospitals.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)

# Getting district codes from the existing hospitals

# Load Saarland NUTS level 3 shapefile
nuts = gpd.read_file('./../index_travel_accessibility/data/raw/NUTS_RG_01M_2021_4326_LEVL_3.shp')
nuts = nuts.to_crs('EPSG:4326')


# Create geometry column from Lat/Lon
# existing_hospitals['geometry'] = existing_hospitals.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)
# existing_hospitals = gpd.GeoDataFrame(existing_hospitals, geometry='geometry', crs='EPSG:4326')

existing_hospitals = gpd.GeoDataFrame(existing_hospitals, geometry='geometry', crs='EPSG:4326')

# Spatial join hospitals with NUTS polygons (only needed columns)
existing_hospitals = gpd.sjoin(
    existing_hospitals,
    nuts[['geometry', 'NUTS_NAME']],
    how='left',
    predicate='within'
)

# Map NUTS_NAME to your official Saarland district codes
name_to_code = {
    "Regionalverband Saarbrücken": "10041",
    "Merzig-Wadern": "10042",
    "Neunkirchen": "10043",
    "Saarlouis": "10044",
    "Saarpfalz-Kreis": "10045",
    "St. Wendel": "10046"
}

# Assign district_code based on district_name
existing_hospitals['district_code'] = existing_hospitals['NUTS_NAME'].map(name_to_code)

# Rename NUTS_NAME to district_name
existing_hospitals.rename(columns={'NUTS_NAME': 'district_name'}, inplace=True)

# Select and reorder columns as needed
existing_hospitals = existing_hospitals[['geometry', 'district_code', 'node', 'bed_allocation', 'Lat', 'Lon']]


existing_hospitals.head()



,geometry,district_code,node,bed_allocation,Lat,Lon
0,POINT (7.04132 49.27707),10041,H1,157.0,49.277066,7.041318
1,POINT (6.95997 49.24756),10041,H2,441.0,49.247555,6.959970
2,POINT (6.99259 49.23708),10041,H3,2.0,49.237085,6.992590
3,POINT (6.99414 49.22122),10041,H4,571.0,49.221221,6.994143
4,POINT (6.96355 49.247),10041,H5,277.0,49.246999,6.963554


In [ ]:
existing_hospitals['type'] = 'existing'
selected_candidates['type'] = 'predicted'

In [13]:
existing_hospitals.head()
selected_candidates.head()

,geometry,district_code,node,bed_allocation,Lat,Lon,type
2,POINT (6.92297 49.29626),10041,264134818,300.0,49.296263,6.922973,predicted
19,POINT (7.01124 49.3396),10043,389033151,300.0,49.339599,7.011240,predicted
21,POINT (7.08335 49.40006),10043,734652583,200.0,49.400060,7.083352,predicted
22,POINT (7.03228 49.35362),10043,1519431680,300.0,49.353624,7.032277,predicted
35,POINT (6.81524 49.33791),10044,318113685,300.0,49.337907,6.815235,predicted


In [ ]:
# If the cols in both exisiting and selected candidates are the same, we can concatenate them
final_results = pd.concat([existing_hospitals, selected_candidates], ignore_index=True)

In [ ]:
final_results.to_excel("results/demand_based_model.xlsx", index=False)